In [ ]:
import pickle 
list_of_dags = pickle.load(open('data/unique_tasks.pkl', 'rb'))
print(f"Number of unique tasks: {len(list_of_dags)}")

In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt

list_of_unique_graphs = []

for dag in list_of_dags:
    G = nx.DiGraph()

    pos = dag['task_dag'].position
    # if 'Start' in pos: pos[0] = pos.pop('Start')
    # if 'Exit' in pos: pos[len(pos)] = pos.pop('Exit')

    for edge in dag['task_dag'].graph.edges:
        G.add_edge(edge[0], edge[1])

    for node in G.nodes:
        G.nodes[node]['pos'] = pos[node]
        G.nodes[node]['type'] = [1,0]
        G.nodes[node]['weight'] = 0

    list_of_unique_graphs.append(G)

print(f"Length of list of unique graphs: {len(list_of_unique_graphs)}")

def draw_graph(graph, show_node_label = False):
    figure = plt.figure(figsize=(10, 5))
    if not show_node_label:
        nx.draw(graph, pos=nx.get_node_attributes(graph, 'pos'), 
                with_labels= not show_node_label,  font_weight='bold', 
                node_color='skyblue', edge_color='gray', 
                node_size=800)

    else: 

        color_map = []
        for node in graph:
            if graph.nodes[node]['type'] == [1,0]:
                color_map.append('skyblue')
            else:
                color_map.append('lightgreen')
        labels = nx.get_node_attributes(graph, 'weight')
        nx.draw_networkx_nodes(graph, pos=nx.get_node_attributes(graph, 'pos'), 
                                node_color=color_map, node_size=800)
        nx.draw_networkx_labels(graph, pos=nx.get_node_attributes(graph, 'pos'), labels=labels)
        nx.draw_networkx_edges(graph, pos=nx.get_node_attributes(graph, 'pos'), edge_color='gray', arrows=True)
    plt.show()


In [ ]:
from tqdm import tqdm

list_of_graph_with_links = []

"""
Add a new node to the graph at the midpoint of each edge
"""
for G in tqdm(list_of_unique_graphs):
    G_copy = G.copy()
    edges = list(G.edges())
    for edge in edges:
        # Create a new node
        new_node = f"{edge[0]}_{edge[1]}"  # create a unique name for the new node

        # Calculate the position of the new node as the midpoint of the two nodes it's between
        pos1 = G.nodes[edge[0]]['pos']
        pos2 = G.nodes[edge[1]]['pos']
        new_pos = ((pos1[0] + pos2[0]) / 2, (pos1[1] + pos2[1]) / 2)

        # Remove the original edge first from the copy
        G_copy.remove_edge(*edge)

        # Add the new node to the graph copy with the calculated position
        G_copy.add_node(new_node, pos=new_pos, type=[0,1], weight=0)

        # Add two new edges to the copy
        G_copy.add_edge(edge[0], new_node)
        G_copy.add_edge(new_node, edge[1])

    # Now G_copy is the modified graph
    list_of_graph_with_links.append(G_copy)


In [ ]:
import random 
import copy

dataset = []

random.seed(200)
"""For each unique dag, we create 10 samples"""
for G in list_of_graph_with_links:
    for i in range(200):
        for node in G:
            if node == 'Start' or node == 'Exit':
                continue
            G.nodes[node]['weight'] = float(random.randint(1, 100))
        dataset.append(copy.deepcopy(G))


def get_paths_duration(graph):
    paths = list(nx.all_simple_paths(graph, source='Start', target='Exit'))
    path_list_with_duration = []
    for path in paths: 
        duration = 0
        for node in path:
            duration += graph.nodes[node]['weight']
        path_list_with_duration.append((path, duration))

    return path_list_with_duration

import torch 
import torch_geometric.data as Data
from torch_geometric.utils import from_networkx

def convert_graph_to_tensor(graph, latency):
    graph_tensor = from_networkx(graph, group_node_attrs=['weight', 'type'])
    graph_tensor.y = torch.tensor([latency])
    graph_tensor.x[:, 0] = graph_tensor.x[:, 0] / 100
    del graph_tensor.pos
    return graph_tensor

print(f"--------------Example One---------------- ")
random_graph = list_of_graph_with_links[random.randint(0, len(list_of_graph_with_links))]
draw_graph(random_graph)
draw_graph(random_graph, show_node_label=True)

path_list = get_paths_duration(random_graph)
for item in path_list: print(item)
max_duration = max(path_list, key=lambda x: x[1])
graph_tensor = convert_graph_to_tensor(random_graph, max_duration[1] )

print(f"Edges: \n{graph_tensor.edge_index}")
print(f"Input feature is {graph_tensor.x}")
print(f"Latency is {graph_tensor.y}")

print(f"--------------Example Two---------------- ")
random_graph = list_of_graph_with_links[random.randint(0, len(list_of_graph_with_links))]
draw_graph(random_graph)
draw_graph(random_graph, show_node_label=True)
path_list= get_paths_duration(random_graph)
for item in path_list: print(item)
max_duration = max(path_list, key=lambda x: x[1])
convert_graph_to_tensor(random_graph, max_duration[1] )

path_list= get_paths_duration(random_graph)
for item in path_list: print(item)
max_duration = max(path_list, key=lambda x: x[1])
graph_tensor = convert_graph_to_tensor(random_graph, max_duration[1] )

print(f"Edges: \n{graph_tensor.edge_index}")
print(f"Input feature is {graph_tensor.x}")
print(f"Latency is {graph_tensor.y}")

In [ ]:
### Creating Dataset 

import os 

directory_path = 'data/unique_graphs_with_links'

if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created successfully.")
else:
    print(f"Directory '{directory_path}' already exists.")

for idx, graph in enumerate(dataset):
    path_list = get_paths_duration(graph)
    max_duration = max(path_list, key=lambda x: x[1])
    graph_tensor = convert_graph_to_tensor(graph, max_duration[1])

    start_index = 0
    if idx > start_index and idx < start_index + 5:
        draw_graph(graph, show_node_label=True)
        for item in path_list: print(item)
        print(f"Latency is {max_duration[1]}")

    torch.save(graph_tensor, f'{directory_path}/graph_{idx}.pt')


In [ ]:
import sys
sys.path.append('../')

In [ ]:
#Inference 
from torch_geometric.utils import to_networkx

def LatNetInference(data):
    model_path = "../gcn/new_model/LatNet_200.pth"
    model = torch.load(model_path,  map_location=torch.device('cpu'))
    pred_latency = model(data.x, data.edge_index, data.batch)
    actual_latency = input.y
    return pred_latency.item(), actual_latency.item()

import os 
entries = os.listdir('data/unique_graphs_with_links')
random_sample = random.sample(entries, 50)

import numpy as np

# Initialize lists to store latencies
pred_latencies = []
actual_latencies = []

for sample in random_sample:
    input = torch.load(f'data/unique_graphs_with_links/{sample}')
    pred_latency, actual_latency = LatNetInference(input)
    # Append latencies to the lists
    pred_latencies.append(pred_latency)
    actual_latencies.append(actual_latency)

# Calculate errors
errors = np.abs(np.array(pred_latencies) - np.array(actual_latencies))

# Create scatter plot
figure = plt.figure(figsize=(12, 5))
plt.scatter(range(len(random_sample)), pred_latencies, label='Predicted Latency', color='blue')
plt.scatter(range(len(random_sample)), actual_latencies, label='Actual Latency', color='red')

# Create a second y-axis
ax2 = plt.twinx()

# Create bar graph on the second y-axis
ax2.bar(range(len(random_sample)), errors, label='Error', color='gray', alpha=0.5)

# Add title and labels
plt.title('Latency Predictions vs Actual')
plt.xlabel('Sample')
plt.ylabel('Latency')
ax2.set_ylabel('Error')

# Add legend
plt.legend(loc='upper left')
ax2.legend(loc='upper right')

# Show the plot
plt.show()